In [9]:
import os
import json

root_dir = '/standard/storage/EgoExoEMS_CVPR2025/Dataset/Final/'

In [20]:
def count_num_bboxes(bbox_data):
    bbox_stats = {}

    for entry in bbox_data:
        # Check if 'bboxes' list is not empty
        if entry["bboxes"]:
            for bbox in entry["bboxes"]:
                obj_class = bbox["class"]
                if(obj_class == "hands"):
                    continue
                if obj_class in bbox_stats:
                    bbox_stats[obj_class] += 1
                else:
                    bbox_stats[obj_class] = 1

    return bbox_stats


In [21]:
import os
import json

# Set to keep track of processed trials
processed_trials = set()
total_bbox_stats = {}

# iterate recursively through all files in the root directory
for root, dirs, files in os.walk(root_dir):
    if "BBOX_MASKS" in root:
        trial_path = "/".join(root.split("/")[:-1])
        
        # Skip this trial if already processed
        if trial_path in processed_trials:
            continue

        print(f"[INFO] Processing bboxes and segmentation masks: {root}")
        print(f"[INFO] Trial path: {trial_path}")

        subject_id = trial_path.split("/")[-3]
        trial_id = trial_path.split("/")[-1]
        print(f"[INFO] Subject ID: {subject_id}")
        print(f"[INFO] Trial ID: {trial_id}")

        bbox_annotation_path = None
        found_bbox_annotations = False  # Flag to stop recursion within the trial

        for file in files:
            if file == "bbox_annotations.json":
                bbox_annotation_path = os.path.join(root, file)
                print(f"[INFO] Found bbox annotation file: {bbox_annotation_path}")

                bbox_data = None

                # Load the json file
                with open(bbox_annotation_path, 'r') as f:
                    bbox_data = json.load(f)

                # Get bbox statistics for the current trial
                bbox_stats = count_num_bboxes(bbox_data)
                print(f"[INFO] Bounding box statistics: {bbox_stats}")

                # Append bbox_stats to total_bbox_stats with subject->trial hierarchy
                if subject_id not in total_bbox_stats:
                    total_bbox_stats[subject_id] = {}
                total_bbox_stats[subject_id][trial_id] = bbox_stats

                found_bbox_annotations = True
                processed_trials.add(trial_path)  # Mark this trial as processed
                break

        # If bbox_annotations is found, skip further directories within this trial
        if found_bbox_annotations:
            del dirs[:]  # Clear dirs to stop os.walk from going deeper in this folder

        print("*" * 10, "=" * 10, "*" * 10)

# Final bbox statistics for all processed trials
print("[INFO] Total bounding box statistics:", total_bbox_stats)


[INFO] Processing bboxes and segmentation masks: /standard/storage/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/1/BBOX_MASKS
[INFO] Trial path: /standard/storage/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/1
[INFO] Subject ID: ng2
[INFO] Trial ID: 1
[INFO] Found bbox annotation file: /standard/storage/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/1/BBOX_MASKS/bbox_annotations.json
[INFO] Bounding box statistics: {'syringe': 3, 'bvm': 116, 'defib pads': 6}
********** ========== **********
[INFO] Processing bboxes and segmentation masks: /standard/storage/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/0/BBOX_MASKS
[INFO] Trial path: /standard/storage/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/0
[INFO] Subject ID: ng2
[INFO] Trial ID: 0
[INFO] Found bbox annotation file: /standard/storage/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/0/BBOX_MASKS/bbox_annotations.json
[INFO] Bounding box statistics: {'bvm': 126, 'defib pads': 31}
********** =======

In [22]:
# Initialize a variable to store the total count of bounding boxes
total_boxes_count = 0

# Iterate through the nested dictionary structure to sum up the counts
for subject_id, trials in total_bbox_stats.items():
    for trial_id, bbox_stats in trials.items():
        # Sum up the count of all bounding boxes in this trial
        total_boxes_count += sum(bbox_stats.values())

print(f"[INFO] Total number of bounding boxes across all subjects: {total_boxes_count}")


[INFO] Total number of bounding boxes across all subjects: 13694
